# Python para Finanças - Detecção de Fraudes

### Importando os dados


Vamos importar a base de dados com transações realizadas por clientes em seus cartões de crédito e identificadas se são ou não fraudulentas para criar o nosso modelo de detecção de fraudes.

In [22]:
# Importando as bibliotecas para leitura e visualização de dados
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [23]:
# Importando a base de dados
df = pd.read_csv('https://raw.githubusercontent.com/afonsosr2/python-financas/main/Video_02/df_balanceado.csv')
df.head()

,gender,city,state,job,category,amt,dia,mes,ano,hora,minuto,is_fraud
0,F,Calhan,CO,Sports development officer,shopping_pos,7.42,22,8,2023,1,17,0.0
1,M,Allison Park,PA,Software engineer,misc_net,74.56,20,6,2023,5,11,0.0
2,F,Bethel,OH,"Teacher, secondary school",home,98.78,9,12,2023,15,11,0.0
3,M,Allison Park,PA,Software engineer,misc_pos,17.03,14,7,2023,21,52,0.0
4,M,Staunton,IL,Editorial assistant,home,45.22,12,12,2023,19,22,0.0


Conseguimos observar que possuímos 12 colunas, sendo 11 delas as características que vamos utilizar para classificar as transações como **genuínas** ou **fraudulentas** e uma delas é a variável resposta ou alvo.

> Dicionário de dados:

* **gender** - sexo biológico do titular do cartão de crédito
* **city** - Cidade do titular do cartão de crédito
* **state** - Estado do titular do cartão de crédito
* **job** - Profissão do titular do cartão de crédito
* **category** - Categoria do comerciante
* **amt** - Valor da transação
* **dia** - dia da transação
* **mes** - mês da transação
* **ano** - ano da transação
* **hora** - hora da transação
* **minuto** - minuto da transação
* **is_fraud** - Indicador de fraude (Classe Alvo)

### Criando nosso modelo de Detecção de fraudes

Para nosso projeto, vamos utilizar a **regressão logística** que é um método estatístico usado para modelar a relação entre uma variável dependente binária (com dois possíveis resultados, como "sim" ou "não") e uma ou mais variáveis independentes.

este modelo estima a probabilidade de um determinado evento ocorrer, ajustando os dados a uma curva logística (função sigmoide). É amplamente utilizado em classificações e previsões, como análise de risco de crédito e detecção de fraudes.

In [24]:
# Importando as bibliotecas necessárias para o modelo
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import numpy as np

#### Separando os dados de treino e teste

Para conseguirmos utilizar a regressão logística vamos precisar transformar os dados que possuímos.

Inicialmente, vamos separar os dados das características (X) da nossa variável alvo (y) e vamos dividir os dados entre treino e teste a fim de treinar o modelo e testá-lo comparando os valores reais aos estimados pelo modelo.

In [25]:
# Separando as features e a variável resposta
X = df.drop(['is_fraud'], axis=1)
y = df['is_fraud']

In [26]:
# Dividindo os dados em bases de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#### Separando as colunas categóricas que vamos aplicar a transformação dos dados e definindo as transformações

Vamos converter as variáveis categóricas em um formato numérico utlizando a técnica do **One Hot Encoding**. Nesta técnica cada categoria é representada como um vetor binário, onde apenas um elemento é "1" (indicando a presença da categoria) e todos os outros são "0".

Por exemplo, para a categoria com os valores "gato" e "cachorro", "gato" seria representado como [1, 0] e "cachorro" como [0, 1].

E, em sequência, vamos partir normalizar as variáveis numéricas utilizando o **StandardScaler()**.

Essa é uma função do Scikit-learn em Python que padroniza (ou normaliza) os dados. Ela transforma os dados de certa forma, que eles tenham média 0 e desvio padrão 1. Esse processo é importante para algoritmos de aprendizado de máquina que são sensíveis à escala dos dados, como é o caso da regressão logística.

In [27]:
# Separando as colunas
colunas_categoricas = ['gender', 'city', 'state', 'category']
colunas_numéricas = ['amt', 'dia', 'mes', 'ano', 'hora', 'minuto']

In [28]:
# Criando os transformadores das colunas categóricas e numéricas
cat = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
num = StandardScaler(with_mean=False)

#### Criando um pipeline de transformação e implementação do modelo de regressão logística

In [29]:
# Criando o ColumnTransformer para aplicar os transformadores
col_transformer = ColumnTransformer( transformers=[
        ('cat', cat, colunas_categoricas),
        ('num', num, colunas_numéricas)
    ])

# Criando um pipeline com o pré-processador e o modelo de regressão logística
model = Pipeline(steps=[
    ('col_transformer', col_transformer),
    ('classificação', LogisticRegression(random_state=42))
])

In [30]:
# Instanciando e treinando o modelo
model.fit(X_train, y_train)

Pipeline(steps=[('col_transformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['gender', 'city', 'state',
                                                   'category']),
                                                 ('num',
                                                  StandardScaler(with_mean=False),
                                                  ['amt', 'dia', 'mes', 'ano',
                                                   'hora', 'minuto'])])),
                ('classificação', LogisticRegression(random_state=42))])

### Testando nosso modelo

Implementamos o nosso modelo, e agora, como ele está funcionando? Estamos tendo bons resultados?

In [31]:
# Gerando previsões com os dados de teste
y_pred= model.predict(X_test)

# Exibindo a acurácia do modelo
print(accuracy_score(y_test, y_pred))

0.849476439790576


O nosso modelo tem a acurácia de aproximadamente 85%. Ou seja, A cada 100 transações ele acerta por volta de 85.

Vamos testar com uma transação fraudulenta para observar se ele consegue acertar?



In [32]:
fraude = pd.DataFrame({'gender': ['M'], 'city': ['Santa Ana'], 'state': ['CA'], 'job': ['Designer, textile'],
                       'category': ['grocery_pos'], 'amt': [107.61], 'dia': [1], 'mes': [2], 'ano': [2024], 'hora': [3],
                       'minuto': [37]})
fraude

,gender,city,state,job,category,amt,dia,mes,ano,hora,minuto
0,M,Santa Ana,CA,"Designer, textile",grocery_pos,107.61,1,2,2024,3,37


In [33]:
fraude_pred = model.predict(fraude)
fraude_pred

array([1.])